In [1]:
import torch
from ucimlrepo import fetch_ucirepo 

from neural_blueprints.utils import Trainer, accuracy, infer_types
from neural_blueprints.config.architectures import TabularBERTConfig
from neural_blueprints.config.utils import TrainerConfig
from neural_blueprints.architectures import TabularBERT
from neural_blueprints.datasets import MaskedTabularDataset, TabularSingleLabelDataset
from neural_blueprints.preprocess import TabularPreprocessor

import logging
logging.basicConfig(
    level=logging.DEBUG,  # or DEBUG if you want even more detail
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [2]:
data = fetch_ucirepo(id=2)
X = data.data.features
y = data.data.targets

data = X.copy()
data['income'] = y['income'].str.replace('.', '')     # Some target values have a trailing dot

dtypes = infer_types(data)
data = data.astype(dtypes)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
preprocessor = TabularPreprocessor()
data, discrete_features, continuous_features = preprocessor.run(data)

2025-12-22 12:57:40,520 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 10 discrete features: ['workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']
2025-12-22 12:57:40,520 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 5 continuous features: ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']


### Income Inference Accuracy

In [4]:
dataset = TabularSingleLabelDataset(
    data=data,
    label_column='income',              # Specify the label column for single-label classification
    discrete_features=discrete_features,
    continuous_features=continuous_features
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [5]:
bert_config = TabularBERTConfig(
    input_cardinalities = dataset.cardinalities,
    output_cardinalities = [3],
    latent_dim = 64,
    encoder_layers=8,
    dropout_p = 0.1,
    normalization = "batchnorm1d",
    activation = "gelu",
    final_activation = None
)

model = TabularBERT(bert_config)
model.blueprint()

TabularBERT(
  (input_projection): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=256, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.0, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=256, out_features=128, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
 

TabularBERTConfig(input_cardinalities=[1, 10, 1, 16, 16, 7, 16, 6, 5, 2, 1, 1, 1, 43], output_cardinalities=[3], latent_dim=64, encoder_layers=8, dropout_p=0.1, normalization='batchnorm1d', activation='gelu', final_activation=None)

In [6]:
trainer = Trainer(
    model=model,
    config=TrainerConfig(
        criterion='cross_entropy',
        optimizer='adam',
        early_stopping_patience=3,
        learning_rate=1e-3,
        weight_decay=1e-5,
        save_weights_path="./models/bert_adult.pth",
        batch_size=128
    )
)
trainer.train(train_dataset, val_dataset, epochs=5, visualize=True)

2025-12-22 12:57:41,775 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:   0%|          | 0/5 [00:00<?, ?epoch/s]2025-12-22 12:57:41,785 - neural_blueprints.architectures.transformer - DEBUG - Input shape: torch.Size([128, 14]), values: tensor([[ 0.4795,  5.0000,  0.2014,  ...,  0.0000,  0.2959,  4.0000],
        [ 0.3699,  5.0000,  0.1928,  ...,  0.0000,  0.3980,  5.0000],
        [ 0.2877,  5.0000,  0.0936,  ...,  0.8655,  0.3980, 40.0000],
        ...,
        [ 0.6986,  1.0000,  0.1858,  ...,  0.3829,  0.3980,  1.0000],
        [ 0.1096,  5.0000,  0.2756,  ...,  0.0000,  0.0714, 40.0000],
        [ 0.4247,  5.0000,  0.0922,  ...,  0.0000,  0.5000, 40.0000]])
2025-12-22 12:57:41,799 - neural_blueprints.architectures.transformer - DEBUG - Positional indices shape: torch.Size([128, 14]), values: tensor([[ 0,  1,  2,  ..., 11, 12, 13],
        [ 0,  1,  2,  ..., 11, 12, 13],
        [ 0,  1,  2,  ..., 11, 12, 13],
        ...,
        [ 0,  1,  2,  ..., 11, 12, 13],
        [ 0,  1,  2,  ..., 11, 12, 13],
        [ 0,  1,  2,  ..., 11, 12, 

TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not list

In [ ]:
X = torch.tensor(val_dataset[:][0])
y = val_dataset[:][1]
with torch.no_grad():
    y_pred = model(X)
    y_pred = y_pred.argmax(dim=1)
print(f"Predictions: {y_pred[:5]}, \n Ground Truth: {y[:5]}")
acc = accuracy(y_pred, y)
print(f"Validation Accuracy: {acc:.4f}")

### Masked Dataset Inference Accuracy

In [ ]:
# Create dataset
dataset = MaskedTabularDataset(
    data, 
    discrete_features, 
    continuous_features,
    mask_prob=0.35
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

2025-12-22 00:13:32,707 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 10 discrete features: ['workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'class']
2025-12-22 00:13:32,707 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 5 continuous features: ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']


In [ ]:
bert_config = TabularBERTConfig(
    input_cardinalities = dataset.cardinalities,   
    latent_dim = 64,
    encoder_layers=8,
    dropout_p = 0.1,
    normalization = "batchnorm1d",
    activation = "gelu",
    final_activation = None
)

model = TabularBERT(bert_config)
model.blueprint()

TabularBERT(
  (input_projection): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=256, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.0, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=256, out_features=128, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
 

TabularBERTConfig(cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42, 2], latent_dim=64, encoder_layers=8, dropout_p=0.1, normalization='batchnorm1d', activation='gelu', final_activation=None)

In [ ]:
trainer = Trainer(
    model=model,
    config=TrainerConfig(
        criterion='mixed_type_reconstruction_loss',
        optimizer='adam',
        early_stopping_patience=3,
        learning_rate=1e-3,
        weight_decay=1e-5,
        save_weights_path="./models/bert_adult.pth",
        batch_size=128
    )
)
trainer.train(train_dataset, val_dataset, epochs=5, visualize=True)

2025-12-22 00:13:44,800 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:  20%|██        | 1/5 [00:34<02:19, 34.94s/epoch]

Epoch 1/5, Training Loss: 33.5590, Validation Loss: 25.5916


Training Epochs:  40%|████      | 2/5 [01:09<01:44, 34.91s/epoch]

...

Epoch 3/5, Training Loss: 24.5489, Validation Loss: 24.2016


Training Epochs:  80%|████████  | 4/5 [02:19<00:34, 34.98s/epoch]

Epoch 4/5, Training Loss: 24.2124, Validation Loss: 24.0425


Training Epochs: 100%|██████████| 5/5 [02:54<00:00, 34.97s/epoch]
2025-12-22 00:16:39,649 - neural_blueprints.utils.trainer - INFO - Training completed in 174.85 seconds.
2025-12-22 00:16:39,649 - neural_blueprints.utils.trainer - INFO - Best validation loss: 2.3718e+01


Epoch 5/5, Training Loss: 23.9635, Validation Loss: 23.7179


In [ ]:
X = val_dataset[:][0]
y = val_dataset[:][1]
mask = val_dataset[:][2]
with torch.no_grad():
    y_pred = model(x=X)

dis_accuracy = 0
cont_accuracy = 0
for column_idx, column_name in enumerate(data.columns):
    print(f"\nFeature Column {column_name}:")
    predicted_attributes = y_pred[column_idx]      # shape: (batch_size, num_classes)
    targets = y[:, column_idx]                     # shape: (batch_size,)

    feature_mask = mask[:, column_idx]                  # shape: (batch_size,)
    predicted_attributes = predicted_attributes[feature_mask]
    if predicted_attributes.size(1) > 1:
        predicted_attributes = predicted_attributes.softmax(dim=-1).argmax(dim=-1).cpu().numpy()
    else:
        predicted_attributes = predicted_attributes.squeeze(-1).cpu().numpy()
    targets = targets[feature_mask].cpu().numpy()

    print("Predicted attribute values:", predicted_attributes[:5])
    print("True attribute values:", targets[:5])

    accuracy_value = accuracy(torch.tensor(predicted_attributes), torch.tensor(targets))
    print(f"Accuracy: {accuracy_value:.4f}")
    if column_name in discrete_features:
        dis_accuracy += accuracy_value
    else:
        cont_accuracy += accuracy_value

avg_dis_accuracy = dis_accuracy / len(discrete_features) if len(discrete_features) > 0 else 0
avg_cont_accuracy = cont_accuracy / len(continuous_features) if len(continuous_features) > 0 else 0
print(f"\nAverage Discrete Accuracy: {avg_dis_accuracy:.4f}")
print(f"Average Continuous Accuracy: {avg_cont_accuracy:.4f}")
avg_accuracy = (dis_accuracy + cont_accuracy) / len(data.columns)
print(f"Overall Average Accuracy: {avg_accuracy:.4f}")


Feature Column age:
Predicted attribute values: [0.31313652 0.18980552 0.04015766 0.579761   0.15299653]
True attribute values: [0.4520548  0.08219178 0.05479452 0.60273975 0.15068494]
Accuracy: 0.2860

Feature Column workclass:
Predicted attribute values: [4 4 4 4 4]
True attribute values: [6. 4. 4. 4. 4.]
Accuracy: 0.6898

Feature Column fnlwgt:
Predicted attribute values: [0.18872398 0.17175908 0.16113535 0.17457661 0.14949259]
True attribute values: [0.31154814 0.08694384 0.11687589 0.01234275 0.01189962]
Accuracy: 0.3201

Feature Column education:
Predicted attribute values: [ 1 15 16 12  2]
True attribute values: [ 1. 15. 16. 12.  2.]
Accuracy: 0.8500

Feature Column education-num:
Predicted attribute values: [16 16 16 16 16]
True attribute values: [16. 16. 16. 16. 16.]
Accuracy: 0.8429

Feature Column marital-status:
Predicted attribute values: [3 3 1 5 5]
True attribute values: [3. 3. 6. 5. 5.]
Accuracy: 0.7911

Feature Column occupation:
Predicted attribute values: [ 1  0  8 